## PART 2: Data Cleaning and Preprocessing

**Scenario:** The first batch of real-time data has arrived from the field. Intelligence reports that a solar storm occurred between 08:00 and 09:30 Zulu, potentially corrupting any data transmitted during that window. Your first task is to isolate and remove these untrustworthy reports. After that, you must develop a robust, reusable Python function to clean the remaining messy data.

**Coding Task Overview:** You will load the messy data, convert the timestamp column to a datetime format, and then filter out the unreliable rows from the solar storm period. You will then develop a `clean_data` function to handle various errors like missing values, incorrect data types, and typos. Finally, you will save both your cleaned data to a new CSV and your reusable function to an external Python utility file.

In [1]:
# Import pandas and numpy
import pandas as pd
import numpy as np
import os

### STEP 1: Load and Inspect the Raw Data

STUDENT CODE REQUIRED

In [2]:
# Load the field_reports_batch_1.csv file into a DataFrame called df_raw.
# Display the .info() summary to show the initial messy state and wrong data types.
DATA_DIR = 'data'

datafilepath = os.path.join(DATA_DIR, 'field_reports_batch_1.csv')

##########################################################
##### START STUDENT CODE HERE:

df_raw = pd.read_csv(datafilepath)
df_raw.info()

##### END STUDENT CODE HERE
##########################################################



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   report_id              200 non-null    int64  
 1   timestamp              200 non-null    object 
 2   team_callsign          200 non-null    object 
 3   latitude               200 non-null    float64
 4   longitude              200 non-null    float64
 5   elevation_m            200 non-null    float64
 6   wind_direction_deg     198 non-null    object 
 7   ambient_temp_c         200 non-null    float64
 8   battery_level_percent  190 non-null    float64
 9   signal_strength        37 non-null     object 
dtypes: float64(5), int64(1), object(4)
memory usage: 15.8+ KB


### STEP 2: Perform Initial Diagnosis on Unclean Data

STUDENT CODE REQUIRED

In [3]:
# Get a count of the different callsigns to see the typos using .value_counts() on the team_callsign column.

##########################################################
##### START STUDENT CODE HERE:

df_raw['team_callsign'].value_counts(dropna=False)

##### END STUDENT CODE HERE
##########################################################




team_callsign
Alpha      10
alfa        8
bravo       6
TEAM001     1
TEAM137     1
           ..
TEAM070     1
TEAM071     1
TEAM073     1
TEAM074     1
TEAM200     1
Name: count, Length: 179, dtype: int64

### STEP 3: Remove Corrupted Data from the Solar Storm Period

STUDENT CODE REQUIRED

To eliminate portions of corrupted data, use ```Pandas``` filtering commands to remove a section of time corresponding to the solar storm.
In ```Pandas```, selecting rows *other than* a certain set can be accomplished using a combination of   ```~``` and
boolean logic operators such as ```&``` using careful placement of parentheses to control the order of operations

Additionally, the ```pandas``` operation ```.copy``` will return a copy of the datafram instead of altering the original

Use ```pd.Timestamp``` to create filtering timestamps to exclude the solar storm period.  Timestamps are in the format ```'YYYY-MM-DD HH:MM:SS'```

* use ```storm_start``` as ```'2025-08-21 08:00:00'```
* and ```storm_end``` as ```'2025-08-21 09:30:00'```

the original dataframe is ```df_raw``` and this code cell should create a filtered dataframe called ```df_filtered``` 



In [ ]:
# Convert the 'timestamp' column to a proper datetime object.
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'], errors='coerce')
before_rows = len(df_raw)

df_filtered = None  #placeholder for the filtered dataframe

##########################################################
##### START STUDENT CODE HERE:


# define the period of the solar storm to be exculded by defining the pd.Timestamp for storm_start and storm_end
# the form of a timestamp is pd.Timestamp('YYYY-MM-DD HH:MM:SS')
storm_start = pd.Timestamp('2025-08-21 08:00:00')
storm_end = pd.Timestamp('2025-08-21 09:30:00')

# Create a copy of a filtered DataFrame that excludes rows during the solar storm period 
# using boolean logic on the timestamps with ~ to exclude a selection and inequality operators such as >=  and <= along with & for boolean AND
# make sure to copy the dataframe int df_filtered instead of altering df_raw
df_filtered = df_raw[~((df_raw['timestamp'] >= storm_start) & (df_raw['timestamp'] <= storm_end))].copy()

##### END STUDENT CODE HERE
##########################################################


after_rows = len(df_filtered)
print(f"Rows before filtering: {before_rows} (should be 200)")
print(f"Rows after filtering: {after_rows} (should be 190)")

Rows before filtering: 200 (should be 200)
Rows after filtering: 190 (should be 190)


### STEP 4: Define the Reusable Cleaning Function

STUDENT CODE REQUIRED

Note:  After you run this cell during Step 5 and confirm it works properly in Step 6, the function you write here should be copied in the python file for utilities which can be imported later.  This file is called ```uav_analysis_tools.py``` and the importable function should be called ```clean_data```.  Don't change other parts of the ```uav_analysis_tools.py``` file

In [5]:
# Define the clean_data function

def clean_data(df, noise_floor=-120):
    """
    Cleans the UAV field report DataFrame.

    This function performs the following steps:
    1.  Standardizes missing value placeholders ('?', 'N/A', '') to np.nan.
    2.  Standardizes 'team_callsign' to lowercase, corrects known typos,
        and fills any missing callsigns with 'unknown'.
    3.  For the 'signal_strength' column, converts it to numeric (coercing errors)
        and imputes missing values (NaNs) with the specified noise_floor.
    4.  For all other numeric columns, converts them to numeric (coercing errors)
        and imputes missing values with the column's median.

    Args:
        df (pd.DataFrame): The raw field report DataFrame.
        noise_floor (int or float, optional): The noise floor (in dBm) to use for
            imputing missing signal strength values. Defaults to -120.

    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    
    # Create a copy to avoid modifying the original DataFrame
    df_clean = df.copy()   

    ##########################################################
    ##### START STUDENT CODE HERE:

    # 1. Standardize missing values across the entire DataFrame using .replace()  you should covert all instances of  ['?', 'N/A', ''] to np.nan.
    df_clean.replace(['?', 'N/A', ''], np.nan, inplace=True)



    # 2. Handle categorical typos (team_callsign)
    if 'team_callsign' in df_clean.columns:
        
        # Fill missing values, standardize casing and strip whitespace using .fillna(), .str.lower(), and .str.strip()
        # Chain string operations for efficiency and clarity
        df_clean['team_callsign'] = df_clean['team_callsign'].fillna('unknown').str.lower().str.strip()

        # Correct known typos such as 'alfa' to 'alpha' by using a map of {`from`: `to`} and the .replace() method
        typo_map = {'alfa': 'alpha'}
        df_clean['team_callsign'] = df_clean['team_callsign'].replace(typo_map)


    # 3. Special handling for signal_strength imputation
    if 'signal_strength' in df_clean.columns:

        # Convert to numeric, coercing errors (like 'ERR-&^%') to NaN by using pd.to_numeric() and the errors='coerce' parameter
        df_clean['signal_strength'] = pd.to_numeric(df_clean['signal_strength'], errors='coerce')

        # Impute NaN values with the noise_floor by using the dataframe .fillna() method, passing in the noise_floor parameter 
        df_clean['signal_strength'] = df_clean['signal_strength'].fillna(noise_floor)



    # 4. Identify and impute other numeric columns with their median
    # Exclude columns we've already handled or know are non-numeric using the dataframe .columns.drop() method and the errors='ignore' parameter
    numeric_cols = df_clean.columns.drop(['timestamp', 'team_callsign', 'report_id', 'signal_strength'], errors='ignore')

    for col in numeric_cols:
        # Convert to numeric, coercing errors to NaN using the pd.to_numeric() function with the errors='coerce' parameter
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        # Calculate the median of the now-numeric column using the dataframe .median() method
        median_val = df_clean[col].median()
        # Fill any NaNs (original or coerced) with the median using the dataframe .fillna() method with inplace=True
        df_clean[col] = df_clean[col].fillna(median_val)



    ##### END STUDENT CODE HERE
    ##########################################################
    
    return df_clean


### STEP 5: Apply the Cleaning Function

STUDENT CODE REQUIRED

Special Note - after confirming the code you wrote for the clean_data() function works properly when it is called from this cell and gives the correct results in step 6, copy the function contents (from the signature line beginning with ```def``` all the way to the end of the ```return``` line) into the ```uav_analysis_tools.py``` file.  Don't change other parts of the ```uav_analysis_tools.py``` file

In [6]:
#from uav_analysis_tools import clean_data

# Apply the clean_data function to the df_filtered DataFrame.
# Store the result in df_clean and display the .info() summary.

df_clean = None #placeholder for cleaned dataframe

##########################################################
##### START STUDENT CODE HERE:

df_clean = clean_data(df_filtered)

##### END STUDENT CODE HERE
##########################################################

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 190 entries, 0 to 199
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   report_id              190 non-null    int64         
 1   timestamp              190 non-null    datetime64[ns]
 2   team_callsign          190 non-null    object        
 3   latitude               190 non-null    float64       
 4   longitude              190 non-null    float64       
 5   elevation_m            190 non-null    float64       
 6   wind_direction_deg     190 non-null    float64       
 7   ambient_temp_c         190 non-null    float64       
 8   battery_level_percent  190 non-null    float64       
 9   signal_strength        190 non-null    float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(1)
memory usage: 16.3+ KB


### STEP 6: Verify the Cleaning Results

You should review the code and ensure that after running it on your ```df_clean``` dataframe, all tests are successful.  If any warnings appear it may be the case that the data was not cleaned properly

Special Note - after confirming the code you wrote for the clean_data() function you wrote in Step 4 works properly when it is called from this cell, copy the function contents (from the signature line beginning with ```def``` all the way to the end of the ```return``` line) into the ```uav_analysis_tools.py``` file where indicated.  Don't change other parts of the ```uav_analysis_tools.py``` file

No Student Code Required for this cell

In [7]:

# 1. Separate columns by their data type
numeric_cols = df_clean.select_dtypes(include=np.number)
categorical_cols = df_clean.select_dtypes(include='object')

# 2. Verify the numeric columns
print("--- Verification of Numeric Data ---")
print("This summary should show a count of 190 for all columns, with no NaNs or extreme outliers.")
display(numeric_cols.describe())


#verify that the meaningful numeric columns contain reasonable values

#verify that latitude is between -90 and 90 and that longitude is between -180 and 180
if 'latitude' in numeric_cols.columns:
    if ((numeric_cols['latitude'] < -90) | (numeric_cols['latitude'] > 90)).any():
        print("Warning: Some latitude entries are out of the valid range (-90 to 90).")
    else:
        print("All latitude entries are within the valid range (-90 to 90).")  
if 'longitude' in numeric_cols.columns:
    if ((numeric_cols['longitude'] < -180) | (numeric_cols['longitude'] > 180)).any():
        print("Warning: Some longitude entries are out of the valid range (-180 to 180).")
    else:
        print("All longitude entries are within the valid range (-180 to 180).")    

#verify that wind direction is between 0 and 360
if 'wind_direction_deg' in numeric_cols.columns:
    if ((numeric_cols['wind_direction_deg'] < 0) | (numeric_cols['wind_direction_deg'] > 360)).any():
        print("Warning: Some wind_direction_deg entries are out of the valid range (0-360).")
    else:
        print("All wind_direction_deg entries are within the valid range (0-360).")

#verify that ambient_temperature is within a reasonable range in celcius
if 'ambient_temperature' in numeric_cols.columns:
    if ((numeric_cols['ambient_temperature'] < -50) | (numeric_cols['ambient_temperature'] > 60)).any():
        print("Warning: Some ambient_temperature entries are outside the reasonable range (-50 to 60 °C).")
    else:
        print("All ambient_temperature entries are within the reasonable range (-50 to 60 °C).")

#verify that battery_level_percent is between 0 and 100%
if 'battery_level_percent' in numeric_cols.columns:
    if ((numeric_cols['battery_level_percent'] < 0) | (numeric_cols['battery_level_percent'] > 100)).any():
        print("Warning: Some battery_level_percent entries are out of the valid range (0-100%).")
    else:
        print("All battery_level_percent entries are within the valid range (0-100%).")

#verify that signal_strength has no values below the noise floor
if 'signal_strength' in numeric_cols.columns:
    if (numeric_cols['signal_strength'] < -120).any():
        print("Warning: Some signal_strength entries are below the noise floor (-120 dBm).")
    else:
        print("All signal_strength entries are above the noise floor (-120 dBm).")  

print("\n" * 2) # Add space for readability

# 3. Verify the categorical columns
print("--- Verification of Categorical Data ---")
print("This summary should show standardized, lowercase callsigns.")
display(categorical_cols.describe())
#show the top 10 most common callsigns to verify no typos
if 'team_callsign' in categorical_cols.columns:
    print("\nTop 10 most common team_callsign values:")
    print(categorical_cols['team_callsign'].value_counts().head(10))    

# 4. Additional checks for specific columns
# check that all team_callsign values are in lowercase and contain no leading/trailing whitespace
if 'team_callsign' in categorical_cols.columns:
    callsign_issues = categorical_cols['team_callsign'].apply(lambda x: x != x.strip() or not x.islower())
    if callsign_issues.any():
        print("Warning: Some team_callsign entries have casing or whitespace issues.")
    else:
        #count the number of unique callsigns
        unique_callsigns = categorical_cols['team_callsign'].nunique()
        print(f"Number of unique team_callsign entries: {unique_callsigns}")
        print("All team_callsign entries are properly formatted.")

--- Verification of Numeric Data ---
This summary should show a count of 190 for all columns, with no NaNs or extreme outliers.


,report_id,latitude,longitude,elevation_m,wind_direction_deg,ambient_temp_c,battery_level_percent,signal_strength
count,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000
mean,104.868421,37.344640,-119.097200,2544.412640,174.784290,24.909122,59.987464,-119.127373
std,56.063631,0.142428,0.168081,630.491722,97.744053,4.965662,23.118881,2.484629
min,1.000000,37.100522,-119.399287,830.012817,0.013178,12.990374,20.054683,-120.000000
25%,58.250000,37.229791,-119.232820,2177.350403,95.448285,21.586530,41.445158,-120.000000
50%,105.500000,37.342196,-119.099862,2611.797974,172.333224,24.776327,59.759282,-120.000000
75%,152.750000,37.460531,-118.958693,2986.838806,258.311969,28.422566,78.447631,-120.000000
max,200.000000,37.598934,-118.801411,3883.077637,350.733465,39.421910,99.927824,-106.447427


All latitude entries are within the valid range (-90 to 90).
All longitude entries are within the valid range (-180 to 180).
All wind_direction_deg entries are within the valid range (0-360).
All battery_level_percent entries are within the valid range (0-100%).
All signal_strength entries are above the noise floor (-120 dBm).



--- Verification of Categorical Data ---
This summary should show standardized, lowercase callsigns.


,team_callsign
count,190
unique,168
top,alpha
freq,18



Top 10 most common team_callsign values:
team_callsign
alpha      18
bravo       6
team129     1
team131     1
team132     1
team133     1
team134     1
team135     1
team137     1
team138     1
Name: count, dtype: int64
Number of unique team_callsign entries: 168
All team_callsign entries are properly formatted.


### STEP 7: Save the Final Cleaned Dataset

No student code required

In [8]:
# Save df_clean to a new CSV file named cleaned_reports_batch_1.csv in the data subdierctory.
# Do not include the pandas index in the saved file.
# Print a confirmation message.
output_path = os.path.join(DATA_DIR, 'cleaned_reports_batch_1.csv')
df_clean.to_csv(output_path, index=False)
print(f"File '{output_path}' has been saved.")

File 'data\cleaned_reports_batch_1.csv' has been saved.
